### In this section we will import required libraries and download S&P_500 data from Yahoo Finace.

---

### Import Libraries and Dependencies

In [1]:
# Import libraries and dependencies
import pandas as pd
import numpy as np
from pathlib import Path
import yfinance as yf
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


Bad key "text.kerning_factor" on line 4 in
C:\Users\eliza\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
#Download stock data
def download_data(tickers, start_date):
    stock_data = yf.download(tickers,start=start_date)
    stock_data = stock_data.drop(columns=["Close", "High", "Low", "Open", "Volume"])
    stock_data["Adj Close"] = stock_data['Adj Close'].astype(float)
    stock_data = stock_data.dropna()
    return stock_data

In [3]:
# Download SPY (SP500) Prices
SP500_df = download_data(["SPY"], "1980-01-01")
SP500_df.head()

[*********************100%***********************]  1 of 1 completed


,Adj Close
Date,
1993-01-29,26.079659
1993-02-01,26.265144
1993-02-02,26.320782
1993-02-03,26.599014
1993-02-04,26.710312


In [4]:
SP500_df.tail()

,Adj Close
Date,
2020-09-17,334.501007
2020-09-18,330.649994
2020-09-21,326.970001
2020-09-22,330.299988
2020-09-23,322.640015


## Save data as a CSV file

In [5]:
csv_path = Path("Data/SP500.csv")
SP500_df .to_csv(csv_path)

## Read in CSV as Pandas DataFrame

In [7]:
# Set path to CSV and read in CSV
csv_path = Path('Data/SP500.csv')
sp500_df=pd.read_csv(csv_path)
#sp500_df